# Introduction

Thank you for your interest in chen and the chennai console. These interactive notebooks introduce chen using the wrapper Python library called `chenpy`. The default docker-compose file from the chen repo starts the chennai console in server mode.

Let's begin by testing the server with a simple `help` command.

In [2]:
from chenpy import client, workspace, utils
from chenpy.detectors import common as atom
from chenpy.logger import console

connection = await client.get("http://chennai:8080", None, "chenadmin", "chenpassword")
res = await client.q(connection, "help")
utils.print_table(res)

val res1: Helper = Welcome to the interactive help system. Below you find a table of all available
top-level commands. To get more detailed help on a specific command, just type

`help.<command>`.

Try `help.importCode` to begin with.
_______________________________________________________________________________________________________________________________________________________________
 command          | description                                                               | example                                                       |
==============================================================================================================================================================|
 annotations      | List annotations                                                          | annotations                                                   |
 callTree         | Show call tree for the given method                                       | callTree(method full name)                                    |
 close            | Close project by name                                                     | close(projectName)                                            |
 cpg              | CPG of the active project                                                 | cpg.method.l                                                  |
 declarations     | List declarations                                                         | declarations                                                  |
 delete           | Close and remove project from disk                                        | delete(projectName)                                           |
 distance         | Show graph edit distance from the source method to the comparison methods | distance(source method iterator, comparison method iterators) |
 exit             | Exit the REPL                                                             |                                                               |
 files            | List files                                                                | files                                                         |
 importAtom       | Create new project from existing atom                                     | importAtom("app.atom")                                        |
 importCode       | Create new project from code                                              | importCode("git url or path")                                 |
 importCpg        | Create new project from existing CPG                                      | importCpg("app.atom")                                         |
 imports          | List imports                                                              | imports                                                       |
 methods          | List methods                                                              | methods('Methods', includeCalls=true, tree=true)              |
 open             | Open project by name                                                      | open("projectName")                                           |
 openForInputPath | Open project for input path                                               |                                                               |
 project          | Currently active project                                                  | project                                                       |
 run              | Run analyzer on active CPG                                                | run.securityprofile                                           |
 save             | Write all changes to disk                                                 | save                                                          |
 sensitive        | List sensitive literals                                                   | sensitive                                                     |
 showSimilar      | Show methods similar to the given method                         

chenpy can interact with chennai server via websockets and seamlessly present the data in the form of tables and trees.

## Analyze a Java application

The best way to learn about chennai is by analyzing a real application. Let's use a simple [vulnerable spring application](https://github.com/HooliCorp/vuln-spring) for this exercise.

There is a convenient `import_code` function in the workspace module for importing code from git and other sources.

In [3]:
await workspace.import_code(connection, "https://github.com/HooliCorp/vuln-spring", project_name="vuln-spring", language="java")

True

### Interacting with the atom

chennai automatically converts the source code into an intermediate representation called the [atom](https://github.com/AppThreat/atom), ready for analysis. Use the `summary` method to view the information about the atom.

In [4]:
console.print(await atom.summary(connection))

Atom Summary      
┏━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Node Type    ┃ Count ┃
┡━━━━━━━━━━━━━━╇━━━━━━━┩
│ Files        │ 4     │
│ Methods      │ 113   │
│ Annotations  │ 37    │
│ Imports      │ 43    │
│ Literals     │ 140   │
│ Config Files │ 2     │
│ Overlays     │ 4     │
└──────────────┴───────┘
      Atom Summary      
┏━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Node Type    ┃ Count ┃
┡━━━━━━━━━━━━━━╇━━━━━━━┩
│ Files        │ 4     │
│ Methods      │ 113   │
│ Annotations  │ 37    │
│ Imports      │ 43    │
│ Literals     │ 140   │
│ Config Files │ 2     │
│ Overlays     │ 4     │
└──────────────┴───────┘

Use the `files` method on the atom to view the files available in the representation.

In [5]:
console.print(await atom.files(connection))

Files                                       
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ File Name                                                       ┃ Method Count ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ src/main/java/com/example/vulnspring/SessionFilter.java         │ 2            │
│ src/main/java/com/example/vulnspring/VulnSpringApplication.java │ 2            │
│ src/main/java/com/example/vulnspring/WebController.java         │ 25           │
└─────────────────────────────────────────────────────────────────┴──────────────┘

`methods` to list the methods along with the filename.

In [6]:
console.print(await atom.methods(connection))

Methods                                                                                                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ File Name                                                       ┃ Methods                                                                                                                                                      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ src/main/java/com/example/vulnspring/SessionFilter.java         │ com.example.vulnspring.SessionFilter.doFilter:void(javax.servlet.ServletRequest,javax.servlet.ServletResponse,javax.servlet.FilterChain)                     │
│                                                                 │ com.example.vulnspring.SessionFilter.<init>:void()                                                                                                           │
├─────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ src/main/java/com/example/vulnspring/VulnSpringApplication.java │ com.example.vulnspring.VulnSpringApplication.main:void(java.lang.String[])                                                                                   │
│                                                                 │ com.example.vulnspring.VulnSpringApplication.<init>:void()                                                                                                   │
├─────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ src/main/java/com/example/vulnspring/WebController.java         │ com.example.vulnspring.WebController.home:java.lang.String(org.springframework.ui.Model,javax.servlet.http.HttpSession)                                      │
│                                                                 │ com.example.vulnspring.WebController.login:java.lang.String(org.springframework.ui.Model)                                                                    │
│                                                                 │ com.example.vulnspring.WebController.login:java.lang.String(javax.servlet.http.HttpSession,java.lang.String,java.lang.String,org.springframework.ui.Model)   │
│                                                                 │ com.example.vulnspring.WebController.loginSuccess:boolean(java.lang.String,java.lang.String)                                                                 │
│                                                                 │ com.example.vulnspring.WebController.logout:java.lang.String(javax.servlet.http.HttpSession)                                                                 │
│                                                                 │ com.example.vulnspring.WebController.update:java.lang.String(javax.servlet.http.HttpSession,org.springframework.ui.Model)                                    │
│                                                                 │ com.example.vulnspring.WebController.update:java.lang.String(javax.servlet.http.HttpSession,java.lang.String,org.springframework.ui.Model)                   │
│                                                                 │ com.example.vulnspring.WebController.checkDB:java.lang.String(java.lang.String,org.springframework.ui.Model)                                                 │
│                      

Library methods accept arguments to customize the behavior. By setting `include_calls` and `tree` parameters, it is possible to visualize the methods information including the calls they make as a tree.

In [7]:
console.print(await atom.methods(connection, include_calls=True, tree=True))

Methods
├── src/main/java/com/example/vulnspring/SessionFilter.java
│   ├── com.example.vulnspring.SessionFilter.doFilter:void(javax.servlet.ServletRequest,javax.servlet.ServletResponse,javax.servlet.FilterChain)
│   │   ├── javax.servlet.FilterChain.doFilter:void(javax.servlet.ServletRequest,javax.servlet.ServletResponse) ⤴
│   │   ├── javax.servlet.FilterChain.doFilter:void(javax.servlet.ServletRequest,javax.servlet.ServletResponse) ⤴
│   │   ├── javax.servlet.http.HttpSession.getAttribute:java.lang.Object(java.lang.String) ⤴
│   │   ├── javax.servlet.http.HttpServletRequest.getMethod:java.lang.String() ⤴
│   │   ├── javax.servlet.http.HttpServletRequest.getServletPath:java.lang.String() ⤴
│   │   ├── java.lang.String.matches:boolean(java.lang.String) ⤴
│   │   ├── javax.servlet.http.HttpServletResponse.sendRedirect:void(java.lang.String) ⤴
│   │   ├── javax.servlet.http.HttpServletRequest.getServletPath:java.lang.String() ⤴
│   │   ├── javax.servlet.http.HttpServletRequest.getSession:javax.servlet.http.HttpSession() ⤴
│   │   └── org.slf4j.Logger.debug:void(java.lang.String) ⤴
│   └── com.example.vulnspring.SessionFilter.<init>:void()
├── src/main/java/com/example/vulnspring/VulnSpringApplication.java
│   ├── com.example.vulnspring.VulnSpringApplication.main:void(java.lang.String[])
│   │   └── org.springframework.boot.SpringApplication.run:org.springframework.context.ConfigurableApplicationContext(java.lang.Class,java.lang.String[]) ⤴
│   └── com.example.vulnspring.VulnSpringApplication.<init>:void()
└── src/main/java/com/example/vulnspring/WebController.java
    ├── com.example.vulnspring.WebController.home:java.lang.String(org.springframework.ui.Model,javax.servlet.http.HttpSession)
    │   ├── org.springframework.ui.Model.addAttribute:org.springframework.ui.Model(java.lang.String,java.lang.Object) ⤴
    │   └── javax.servlet.http.HttpSession.getAttribute:java.lang.Object(java.lang.String) ⤴
    ├── com.example.vulnspring.WebController.login:java.lang.String(org.springframework.ui.Model)
    ├── com.example.vulnspring.WebController.login:java.lang.String(javax.servlet.http.HttpSession,java.lang.String,java.lang.String,org.springframework.ui.Model)
    │   ├── org.slf4j.Logger.debug:void(java.lang.String) ⤴
    │   ├── com.example.vulnspring.WebController.loginSuccess:boolean(java.lang.String,java.lang.String)
    │   ├── org.slf4j.Logger.debug:void(java.lang.String) ⤴
    │   └── javax.servlet.http.HttpSession.setAttribute:void(java.lang.String,java.lang.Object) ⤴
    ├── com.example.vulnspring.WebController.loginSuccess:boolean(java.lang.String,java.lang.String)
    │   ├── org.springframework.jdbc.core.JdbcTemplate.queryForMap:java.util.Map(java.lang.String) ⤴
    │   └── java.util.Map.containsKey:boolean(java.lang.Object) ⤴
    ├── com.example.vulnspring.WebController.logout:java.lang.String(javax.servlet.http.HttpSession)
    │   └── javax.servlet.http.HttpSession.invalidate:void() ⤴
    ├── com.example.vulnspring.WebController.update:java.lang.String(javax.servlet.http.HttpSession,org.springframework.ui.Model)
    │   ├── org.springframework.ui.Model.addAttribute:org.springframework.ui.Model(java.lang.String,java.lang.Object) ⤴
    │   ├── org.springframework.jdbc.core.JdbcTemplate.queryForMap:java.util.Map(java.lang.String,java.lang.Object[]) ⤴
    │   ├── java.util.Map.get:java.lang.Object(java.lang.Object) ⤴
    │   └── javax.servlet.http.HttpSession.getAttribute:java.lang.Object(java.lang.String) ⤴
    ├── com.example.vulnspring.WebController.update:java.lang.String(javax.servlet.http.HttpSession,java.lang.String,org.springframework.ui.Model)
    │   ├── org.springframework.ui.Model.addAttribute:org.springframework.ui.Model(java.lang.String,java.lang.Object) ⤴
    │   ├── org.slf4j.Logger.info:void(java.lang.String) ⤴
    │   ├── javax.servlet.http.HttpSession.getAttribute:java.lang.Object(java.lang.String) ⤴
    │   ├── org.slf4j.Logger.info:void(java.lang.String) ⤴
    │   ├── org.springframework.jdbc.core.Jdb

Let's try the other methods such as `annotations`, `imports`, `declarations`, `sensitive`.

In [8]:
console.print(await atom.annotations(connection))

Annotations                                                                                                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ File Name                                               ┃ Methods                                                                                                                                               ┃ Annotations                                              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ src/main/java/com/example/vulnspring/SessionFilter.java │ com.example.vulnspring.SessionFilter.doFilter:void(javax.servlet.ServletRequest,javax.servlet.ServletResponse,javax.servlet.FilterChain)              │ <unresolvedNamespace>.Override                           │
├─────────────────────────────────────────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────┤
│ src/main/java/com/example/vulnspring/WebController.java │ com.example.vulnspring.WebController.home:java.lang.String(org.springframework.ui.Model,javax.servlet.http.HttpSession)                               │ org.springframework.web.bind.annotation.GetMapping       │
├─────────────────────────────────────────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────┤
│ src/main/java/com/example/vulnspring/WebController.java │ com.example.vulnspring.WebController.login:java.lang.String(org.springframework.ui.Model)                                                             │ org.springframework.web.bind.annotation.GetMapping       │
├─────────────────────────────────────────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────┤
│ src/main/java/com/example/vulnspring/WebController.java │ com.example.vulnspring.WebController.login:java.lang.String(javax.servlet.http.HttpSession,java.lang.String,java.lang.String,org.springframework.ui.… │ org.springframework.web.bind.annotation.PostMapping      │
├─────────────────────────────────────────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────┤
│ src/main/java/com/example/vulnspring/WebController.java │ com.example.vulnspring.WebController.logout:java.lang.String(javax.servlet.http.HttpSession)                                                          │ org.springframework.web.bind.annotation.GetMapping       │
├─────────────────────────────────────────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────┤
│ src/main/java/com/example/vulnspring/WebController.java │ com.example.vulnspring.WebController.update:java.lang.String(javax.servlet.http.HttpSession,org.springframework.ui.Model)                             │ org.springframework.web.bind.annotation.GetMapping       │
├─────────────────────────────────────────────────────────┼─────

In [9]:
console.print(await atom.imports(connection))

Imports                                                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ File Name                                                       ┃ Import                                                       ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ src/main/java/com/example/vulnspring/SessionFilter.java         │ java.io.IOException                                          │
├─────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────┤
│ src/main/java/com/example/vulnspring/SessionFilter.java         │ javax.servlet.Filter                                         │
├─────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────┤
│ src/main/java/com/example/vulnspring/SessionFilter.java         │ javax.servlet.FilterChain                                    │
├─────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────┤
│ src/main/java/com/example/vulnspring/SessionFilter.java         │ javax.servlet.ServletException                               │
├─────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────┤
│ src/main/java/com/example/vulnspring/SessionFilter.java         │ javax.servlet.ServletRequest                                 │
├─────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────┤
│ src/main/java/com/example/vulnspring/SessionFilter.java         │ javax.servlet.ServletResponse                                │
├─────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────┤
│ src/main/java/com/example/vulnspring/SessionFilter.java         │ javax.servlet.annotation.WebFilter                           │
├─────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────┤
│ src/main/java/com/example/vulnspring/SessionFilter.java         │ javax.servlet.http.HttpServletRequest                        │
├─────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────┤
│ src/main/java/com/example/vulnspring/SessionFilter.java         │ javax.servlet.http.HttpServletResponse                       │
├─────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────┤
│ src/main/java/com/example/vulnspring/SessionFilter.java         │ org.slf4j.Logger                                             │
├─────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────┤
│ src/main/java/com/example/vulnspring/SessionFilter.java         │ org.slf4j.LoggerFactory                                      │
├─────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────┤
│ src/main/java/com/example/vulnspring/SessionFilter.java         │ org.springframework.stereotype.Component                     │
├─────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────┤
│ src/main/java/com/example/vulnspring/VulnSpringApplication.java │ org.springframework.boot.SpringApplication                   │
├─────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────┤
│ src/main/java/com/example/vulnspring/VulnSpringApplication.java │ org.springframework.boot.autoconfigure.SpringBootApplication │


In [10]:
console.print(await atom.declarations(connection))

Declarations                                    
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ File Name                                                       ┃ Declarations  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ src/main/java/com/example/vulnspring/SessionFilter.java         │ request       │
│                                                                 │ response      │
│                                                                 │ chain         │
├─────────────────────────────────────────────────────────────────┼───────────────┤
│ src/main/java/com/example/vulnspring/VulnSpringApplication.java │ args          │
├─────────────────────────────────────────────────────────────────┼───────────────┤
│ src/main/java/com/example/vulnspring/WebController.java         │ body          │
│                                                                 │ newName       │
│                                                                 │ amount        │
│                                                                 │ dbpath        │
│                                                                 │ streetNumber  │
│                                                                 │ session       │
│                                                                 │ node          │
│                                                                 │ model         │
│                                                                 │ toAccount     │
│                                                                 │ currentString │
│                                                                 │ username      │
│                                                                 │ streetName    │
│                                                                 │ jwtString     │
│                                                                 │ desk          │
│                                                                 │ encodedString │
│                                                                 │ password      │
└─────────────────────────────────────────────────────────────────┴───────────────┘

In [11]:
console.print(await atom.sensitive(connection))

Sensitive                                        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ File Name                                                       ┃ Sensitive Literals ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ src/main/java/com/example/vulnspring/SessionFilter.java         │ N/A                │
├─────────────────────────────────────────────────────────────────┼────────────────────┤
│ src/main/java/com/example/vulnspring/VulnSpringApplication.java │ N/A                │
├─────────────────────────────────────────────────────────────────┼────────────────────┤
│ src/main/java/com/example/vulnspring/WebController.java         │ N/A                │
└─────────────────────────────────────────────────────────────────┴────────────────────┘

### Similarity analysis

Out of the box, chennai comes with support for frameworks such as NetworkX and PyTorch. Let's try a simple analysis to start with - looking for methods that are structurally similar to the input method.

In [12]:
console.print(await atom.show_similar(connection, method_fullname="com.example.vulnspring.WebController.login:java.lang.String(org.springframework.ui.Model)", compare_pattern="com.example.vulnspring.WebController"))

Similarity analysis for `com.example.vulnspring.WebController.login:java.lang.String(org.springframework.ui.Model)`                                                           
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ File Name                                               ┃ Method Name                                                                                                                                                ┃ Edit Distance ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ src/main/java/com/example/vulnspring/WebController.java │ com.example.vulnspring.WebController.<init>:void()                                                                                                         │ 29.0          │
├─────────────────────────────────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼───────────────┤
│ src/main/java/com/example/vulnspring/WebController.java │ com.example.vulnspring.WebController.issue:java.lang.String(org.springframework.ui.Model)                                                                  │ 51.0          │
├─────────────────────────────────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼───────────────┤
│ src/main/java/com/example/vulnspring/WebController.java │ com.example.vulnspring.WebController.checkDB:java.lang.String()                                                                                            │ 60.0          │
├─────────────────────────────────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼───────────────┤
│ src/main/java/com/example/vulnspring/WebController.java │ com.example.vulnspring.WebController.logout:java.lang.String(javax.servlet.http.HttpSession)                                                               │ 68.0          │
├─────────────────────────────────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼───────────────┤
│ <empty>                                                 │ com.example.vulnspring.WebController.<clinit>:void()                                                                                                       │ 75.0          │
├─────────────────────────────────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼───────────────┤
│ src/main/java/com/example/vulnspring/WebController.java │ com.example.vulnspring.WebController.support:java.lang.String(org.springframework.ui.Model)                                                                │ 82.0          │
├─────────────────────────────────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼───────────────┤
│ src/main/java/com/example/vulnspring/WebController.java │ com.example.vulnspring.WebController.home:java.lang.String(org.springframework.ui.Model,javax.servlet.http.HttpSession)                                    │ 105.0         │
├─────────────────────────────────────────────────────────┼──────────────────────────────────────

## Conclusion

This notebook introduced the chennai server and some core methods available for interacting with code bases.